In [ ]:
import pandas as pd
import six
from google.cloud import translate_v2 as translate
import os

credential_path = ""
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path


In [ ]:
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)
    # print(u"Text: {}".format(result["input"]))
    # print(u"Translation: {}".format(result["translatedText"]))
    # print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
    return result["translatedText"]


In [ ]:
PATHS = [
    "../data/arabic/test-gold/covid19_disinfo_binary_arabic_test_gold.tsv",
    "../data/arabic/test-input/covid19_disinfo_binary_arabic_test_input.tsv",
    "../data/arabic/v3/covid19_disinfo_binary_arabic_dev.tsv",
    "../data/arabic/v3/covid19_disinfo_binary_arabic_train.tsv",
    "../data/bulgarian/test-gold/covid19_disinfo_binary_bulgarian_test_gold.tsv",
    "../data/bulgarian/test-input/covid19_disinfo_binary_bulgarian_test_input.tsv",
    "../data/bulgarian/v1/covid19_disinfo_binary_bulgarian_dev.tsv",
    "../data/bulgarian/v1/covid19_disinfo_binary_bulgarian_train.tsv",
]

for path in PATHS:
    df = pd.read_csv(path, sep="\t", header=None, skiprows=1)
    if len(df.columns) > 8:
        df = df.rename(
            {
                0: "tweet_no",
                1: "tweet_text",
                2: "q1_label",
                3: "q2_label",
                4: "q3_label",
                5: "q4_label",
                6: "q5_label",
                7: "q6_label",
                8: "q7_label"
            },
            axis="columns",
        )
    else:
        df = df.rename(
            {
                0: "tweet_no",
                1: "tweet_text"
            }
        )
    df["tweet_text_en"] = df.tweet_text.apply(lambda x: translate_text("en", x))
    df.to_csv(path, sep="\t", index=False)
